In [2]:
import os
import time
import numpy as np
import sort
from sort import *

%load_ext line_profiler
%load_ext cython

In [7]:
# all train
sequences = ['PETS09-S2L1','TUD-Campus','TUD-Stadtmitte','ETH-Bahnhof','ETH-Sunnyday','ETH-Pedcross2','KITTI-13','KITTI-17','ADL-Rundle-6','ADL-Rundle-8','Venice-2']

if not os.path.exists('output'):
    os.makedirs('output')

In [8]:
def run(total_time=0.0, total_frames=0):
    for seq in sequences:
        mot_tracker = Sort() #create instance of the SORT tracker
        seq_dets = np.loadtxt('data/%s/det.txt'%(seq),delimiter=',').astype('float32') #load detections
        with open('output/%s.txt'%(seq),'w') as out_file:
            print("Processing %s."%(seq))
            for frame in range(int(seq_dets[:,0].max())):
                frame += 1 #detection and frame numbers begin at 1
                dets = seq_dets[seq_dets[:, 0] == frame, 2:7]
                dets[:,2:4] += dets[:, 0:2] #convert to [x1,y1,w,h] to [x1,y1,x2,y2]
                total_frames += 1

                start_time = time.time()
                trackers = mot_tracker.update(dets)
                cycle_time = time.time() - start_time
                total_time += cycle_time

                for d in trackers:
                    print('%d,%d,%.2f,%.2f,%.2f,%.2f,1,-1,-1,-1'%(frame,d[4],d[0],d[1],d[2]-d[0],d[3]-d[1]),file=out_file)

    print("Total Tracking took: %.3f for %d frames or %.1f FPS"%(total_time,total_frames,total_frames/total_time))

In [9]:
# lprun -f KalmanBoxTracker.predict -f KalmanBoxTracker.__init__ -f Sort.update -f associate_detections_to_trackers -f run run(total_time, total_frames)
run()

Processing PETS09-S2L1.
Processing TUD-Campus.
Processing TUD-Stadtmitte.
Processing ETH-Bahnhof.
Processing ETH-Sunnyday.
Processing ETH-Pedcross2.
Processing KITTI-13.
Processing KITTI-17.
Processing ADL-Rundle-6.
Processing ADL-Rundle-8.
Processing Venice-2.
Total Tracking took: 18.740 for 5500 frames or 293.5 FPS
